In [2]:
!pip install pandas openpyxl xlrd

  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.3.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl (11.3 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached numpy-2.3.4-cp311-cp311-win_amd64.whl (13.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   ---------------------------------------- 0/7 [pytz]
   ---------------------------------------- 0/7 [pytz]
   ----- ---------------------------------- 1/7 [xlrd]
   ----- ---------------------------------- 1/7 [xlrd]
   ----------- ------

In [3]:
import pandas as pd
import re
import os
from IPython.display import display

#  Ruta donde están los archivos en formato .xls
carpeta_xls = "data"
archivos = [f for f in os.listdir(carpeta_xls) if f.endswith(".xls")]

# Se crea una lista para almacenar todos los registros verticales
registros_verticales = []

# Se procesa cada uno de los archivos
for archivo in archivos:
    ruta = os.path.join(carpeta_xls, archivo)
    nombre_archivo = os.path.splitext(archivo)[0]

    # Se extrae la fecha de cada toma en formato MM/YYYY
    fecha_match = re.search(r"(\d{2})_(\d{4})", nombre_archivo)
    if fecha_match:
        mes, año = fecha_match.groups()
        fecha_columna = f"{mes}/{año}"
    else:
        fecha_columna = nombre_archivo

    try:
        df = pd.read_excel(ruta, sheet_name=0, engine='xlrd')
        df.columns = [str(col).strip() for col in df.columns]

        for i in range(0, len(df.columns), 3):
            grupo = df.iloc[:, i:i+3].copy()
            grupo.columns = ["Celda N°", "Tensión (V)", "Gravedad Específica"]
            grupo = grupo[grupo["Celda N°"].apply(lambda x: str(x).strip().isdigit())]

            grupo["Celda N°"] = pd.to_numeric(grupo["Celda N°"], errors="coerce")
            grupo["Tensión (V)"] = grupo["Tensión (V)"].astype(str).str.replace(",", ".").astype(float)
            grupo["Gravedad Específica"] = pd.to_numeric(grupo["Gravedad Específica"], errors="coerce")
            grupo["Fecha"] = fecha_columna

            grupo = grupo[grupo["Celda N°"].between(1, 60)]
            registros_verticales.append(grupo)

    except Exception as e:
        print(f" Error procesando {archivo}: {e}")

#  Se unen todos los registros en forma vertical
df_vertical = pd.concat(registros_verticales, ignore_index=True)

#  Ordenar por No. de  celda N° y fecha
df_vertical["Fecha_orden"] = pd.to_datetime("01/" + df_vertical["Fecha"], format="%d/%m/%Y", errors="coerce")
df_vertical = df_vertical.sort_values(["Fecha_orden", "Celda N°"]).drop(columns="Fecha_orden").reset_index(drop=True)

#  Se muestran los resultados

display(df_vertical)


,Celda N°,Tensión (V),Gravedad Específica,Fecha
0,1,2.2,1285.0,12/2020
1,2,2.3,1280.0,12/2020
2,3,2.3,1280.0,12/2020
3,4,2.2,1275.0,12/2020
4,5,2.2,1275.0,12/2020
...,...,...,...,...
1615,56,2.1,1275.0,08/2025
1616,57,2.1,1275.0,08/2025
1617,58,2.1,1275.0,08/2025
1618,59,2.1,1275.0,08/2025


In [5]:
df_vertical.describe()

,Celda N°,Tensión (V),Gravedad Específica
count,1620.00000,1620.000000,1500.000000
mean,30.50000,2.181543,1275.680838
std,17.32345,0.061874,6.237875
min,1.00000,2.100000,1235.000000
25%,15.75000,2.100000,1275.000000
50%,30.50000,2.200000,1275.000000
75%,45.25000,2.200000,1275.000000
max,60.00000,2.300000,1300.000000


In [9]:
df_vertical_grouped = df_vertical.groupby("Celda N°").count().reset_index()
print(df_vertical_grouped)

    Celda N°  Tensión (V)  Gravedad Específica  Fecha
0          1           27                   25     27
1          2           27                   25     27
2          3           27                   25     27
3          4           27                   25     27
4          5           27                   25     27
5          6           27                   25     27
6          7           27                   25     27
7          8           27                   25     27
8          9           27                   25     27
9         10           27                   25     27
10        11           27                   25     27
11        12           27                   25     27
12        13           27                   25     27
13        14           27                   25     27
14        15           27                   25     27
15        16           27                   25     27
16        17           27                   25     27
17        18           27   